## Data Access & Pre-processing

This notebook is a walk-through of the following steps:
- Accessing GEBCO bathymetry data, clipping that data to a region of interest, and masking that data to exclude values above sea level.
- Run the Benthic Terrain Modeler
- Accessing Bio-ORACLE and NEMO-MEDUSA raster layers

### <u>GEBCO Bathymetry Data<u>

#### 1. Download GEBCO bathymetry data

The GEBCO_2023 Grid (sub-ice topo/bathy) can be downloaded at [this link](https://www.gebco.net/data_and_products/gridded_bathymetry_data/) in several formats: NetCDF, EsrI ACRII raster, or GeoTIFF. Download the "netCDF" file. Note that this file will take several minutes to download.

<img src="images/gebcodownload.png" style="height:250px" />

#### 2. Define your region of interest

In the **Analysis** tab, select "Environments" in the Geoprocessing section. In the "Processing Extent", select "As Specified Below", and input the coordinates that you are interested in. 
- need input from Keith here - how to set Extent and have it saved. I think what I'm doing is right but not sure.

<img src="images/setextent.png" style="height:400px" />

#### 3. Make Multidimensional Raster Layer

Use the **Make Multidimensional Raster Layer** tool, with the netCDF file as your input file. Select the variable that you want to use in your layer (elevation). Since your extent has already been defined, the coordinates of the extent will automatically populate. You shoudl now be able to visualize the GEBCO output for your region of interest.

<img src="images/makemulti.png" style="height:300px" /> <img src="images/makemulti_westcoast.png" style="height:300px" />

#### 4. Remove values above sea-level

Use the **Con** Spatial Analyst tool to make a conditional statement on each of the raster grid cells. Set the Input conditional raster as the raster made in the previous step, and set the condition to "Where Values is less than 0". <br>
Use the same raster layer as the "Input true raster". What this does is it will use grid cells from the GEBCO raster if the grid cell values from the same raster are less than 0. <br> 
Save the output raster as a file with a ".crf" extension. This is a file format that ESRI uses to support multidimensional data. <br>
You should now be able to see a new layer with values only below sea level.

<img src="images/con.png" style="height:300px" /> <img src="images/con_westcoast.png" style="height:300px" />

#### 5. Convert raster to TIFF

Next we can convert theh CRF file to TIFF format. This step will ensure that all of the layers we are using are in the same format. 
Use the **Copy Raster** tool with the .crf file from the previous step as the input raster, and set the output raster dataset as a file with a ".tif" extension. Under "Format", select "TIFF Format". Keep the NoData value as the default value (which should be -32718)

### <u>Running the Benthic Terrain Modeler<u>

We will use version 3.0 of the Benthic Terrain Modeler (BTM) for ArcGIS Pro. This tool is used to classify and analyze benthic terrain for geomorphology and classification. <br>
Here are some useful links if you want to learn more about how the BTM works: <br>
- [Walbridge et al., 2018](https://www.mdpi.com/2076-3263/8/3/94)
- [Software ReadME](https://dusk.geo.orst.edu/djl/samoa/tools/readme.txt) <br>


##### <u>Steps for installing the BTM Toolbox onto ArcGIS Pro:<u>
1. Download the BTM Version 3.0 Toolbox by clicking "Download" at the top right of [this page](https://www.arcgis.com/home/item.html?id=b0d0be66fd33440d97e8c83d220e7926#!?TB_iframe=true)
2. Extract the contents of the zipped folder on your local machine
3. In ArcGIS Pro, go to View -> Catalog Pane -> Right-Click on Toolboxes -> Select "Add Toolbox"
4. Navigate to the extracted BTM toolbox, and select the "btm.pyt" toolbox, and click OK. You should now see the toolbox appear in your list of toolboxes. 

##### <u>Steps for running the BTM:<u>
1. Under the Bathymetric Position Index (BPI) toolset, select the script "Build Broad Scale BPI". Set the inner radius to 1 and an outer radius to 10, giving a scale factor of ∼100 km. <br>
2. Under the Bathymetric Position Index (BPI) toolset, select the script "Build Fine Scale BPI". Set the inner radius of 1 and an outer radius of 2, giving a scale factor of ∼20 km. <br>
3. Under the Surface Deviations and Statistics toolset, select the script "Calculate Slope". <br>
For all of these steps, the input bathymetry layer should be the GEBCO bathymetry raster layer which has been clipped and masked.

<img src="images/bbpi.png" style="height:400px" />

### <u>Bio-ORACLE Environmental Data<u>

#### Download rasters of global benthic mean temperature and salinity estimates from the [Bio-ORACLE download manager](https://www.bio-oracle.org/downloads-to-email.php)
The following datasets were downloaded as TIFF files:
- benthic temperature (present)
- benthic salinity (present) 

<img src="images/biooracle1.png" style="height:400px" /> <img src="images/biooracle2.png" style="height:200px" />

### <u>NEMO-MEDUSA Benthic POC Flux Model<u>

#### Next, access the NEMO-MEDUSA Seafloor Particular Organic Carbon Flux model output from [Zenodo](https://zenodo.org/record/6513616).
- Download the file named MEDUSA_ORCA0083_REGRID_2006-2015_flux.nc <br>
- There are a few steps that need to be taken to be able to work with this netCDF file for our analyses:


1. Use the **Make Multidimensional Raster Layer** tool, using the netcdf file as the Input Multidimensional Raster, and selecting "avg_flux" as the variable to use in the raster layer. The Dimension Information and Extent does not need to be changed. <br>

<img src="images/makemultidim.png" style="height:450px" />

2. Use the **Copy Raster** tool to make a copy of the mutlidimensional raster and save is as a (Cloud Raster Format (CRF)), selecting the "Process as Multidimensional" option.

<img src="images/copyraster.png" style="height:450px" />

3. Rescale the resolution of the multidimensional raster by specifying the x and y scale factors using the **Rescale** tool. Set the x and y scale factor to 8.333333333E-02 (1/12 of a degree). Make sure to use the CRF file from the previous step in the rescaling. 

<img src="images/rescale.png" style="height:250px" />

4. Use the **Shift** tool to move the x and y values of the raster to new geographic location - this will ensure that this layer can be perfectly overlaid with the other layers.
Set the following shifts:
- Shift x-coordinates by: -180
- Shift y-coordinates by: 2070

<img src="images/shift.png" style="height:250px" />

5. Lastly, use the **Define Projection** tool to set the projection to "GCS_WGS_1984" to ensure the layer is in the same projection as the other raster layers used in this analysis. Once that step has been run, the avg benthic POC flux model output should be properly projected on a map: 

<img src="images/poc_raster.png" style="height:350px" />